In [ ]:
import tkinter as tk
import random
import math

PLAYER_X = 'X'
PLAYER_O = 'O'
EMPTY = ' '
ROWS = 6
COLUMNS = 7
SIMULATION_COUNT = 1000

In [ ]:
class CMTS:
    #constructor of CMTS class
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.wins = 0
        self.visits = 0

    #checks if it's full expended(all moves are explored)
    def is_fully_expanded(self):
        return len(self.children) == len(self.state.get_legal_moves())

    #selects the child node to explore based on UCT
    def select_child(self, exploration_factor=1.414):
        best_child = None
        best_score = -float('inf')
        for child in self.children:
            if child.visits == 0:
                score = -float('inf')
            else:
                score = child.wins / child.visits + exploration_factor * math.sqrt(2 * math.log(self.visits) / child.visits)
            if score > best_score:
                best_score = score
                best_child = child
        return best_child

    #expands the current node by creating new child nodes for each possible move in the current state
    def expand(self):
        legal_moves = self.state.get_legal_moves()
        for move in legal_moves:
            new_state = self.state.copy()
            new_state.make_move(move)
            new_child = CMTS(new_state, parent=self)
            self.children.append(new_child)
        return random.choice(self.children) if self.children else None

    #backpropagates the result of a simulation up the tree search
    def backpropagate(self, result):
        self.visits += 1
        self.wins += result
        if self.parent:
            self.parent.backpropagate(result)

##O objetivo desta classe é representar um nó numa árvore de busca utilizada no algoritmo MCTS. Cada nó armazena um estado do jogo e contadores que são usados na seleção e atualização dos nós durante o processo de busca.Oque é o MCTS? O MCTS é um algoritmo de busca probabilístico usado em jogos e outros problemas de decisão, onde a decisão final é baseada em simulações de jogadas possíveis.


---


*   **__init__**: Método especial de inicialização da classe, responsável por configurar o jogo quando uma instância da classe é criada. Ele define as configurações iniciais da janela do jogo, cria os botões do tabuleiro, e inicializa variáveis como o estado atual do tabuleiro e o jogador atual.
*   **is_fully_expanded**: Este método verifica se o nó está totalmente expandido, ou seja, se todos os movimentos possíveis foram explorados e geraram outros nós. Retorna "True" se o número de jogadas é igual ao número de movimentos possíveis a partir do estado atual.
*   **select_child**: Este método seleciona o melhor movimento do nó atual com base numa fórmula de seleção do algoritmo MCTS. Calcula um valor de pontuação para cada movimento usado.Retorna o melhor movimento com base na pontuação calculada.
*   **expand**: Este método expande o nó atual, gerando nós "filhos" para cada movimento possível a partir do estado atual do jogo. Cada "filho" é criado a partir de um novo estado resultante do movimento e adicionado à lista de "filhos" do nó atual. Retorna um dos "filhos" aleatoriamente, se houver, ou "None" se não houver "filhos".
*   **backpropagate**: Este método atualiza os contadores de vitórias e visitas do nó atual e, de seguida, chama recursivamente a função backpropagate() no seu nó principal, se houver, para continuar a retropropagação dos resultados até a raiz da árvore de busca.











In [ ]:
class GameState:
    #constructor of GameState class
    def __init__(self, board, current_player):
        self.board = board
        self.current_player = current_player

    #list of legal moves for the current player
    def get_legal_moves(self):
        legal_moves = []
        for col in range(COLUMNS):
            if self.board[0][col] == EMPTY:
                legal_moves.append(col)
        return legal_moves

    #makes a move for the current player at the specified column
    def make_move(self, col):
        for row in range(ROWS-1, -1, -1):
            if self.board[row][col] == EMPTY:
                self.board[row][col] = self.current_player
                break
        self.current_player = PLAYER_X if self.current_player == PLAYER_O else PLAYER_O

    #checks if there is a winner
    def check_winner(self):
        # Horizontal
        for row in self.board:
            for i in range(COLUMNS - 3):
                if all(piece == self.current_player for piece in row[i:i+4]):
                    return True

        # Vertical
        for col in range(COLUMNS):
            for i in range(ROWS - 3):
                if all(self.board[row][col] == self.current_player for row in range(i, i+4)):
                    return True

        # Diagonal (left to right)
        for row in range(ROWS - 3):
            for col in range(COLUMNS - 3):
                if all(self.board[row+i][col+i] == self.current_player for i in range(4)):
                    return True

        # Diagonal (right to left)
        for row in range(ROWS - 3):
            for col in range(3, COLUMNS):
                if all(self.board[row+i][col-i] == self.current_player for i in range(4)):
                    return True

        return False

    #checks if the board is full
    def is_board_full(self):
        return all(piece !=EMPTY for row in self.board for piece in row)

    #returns a copy of the current game
    def copy(self):
        return GameState([row[:] for row in self.board], self.current_player)


# A classe GameState representa o estado atual do jogo Connect Four. Ela armazena informações sobre o tabuleiro de jogo e o jogador atual.


---


*   **init**: Este é o método de inicialização da classe. Ele recebe o estado atual do tabuleiro (`board`) e o jogador que está a jogar (`current_player`) e armazena como atributos do objeto `GameState`.
*   **get_legal_moves**: Este método vai retornar uma lista de movimentos possíveis disponíveis para o jogador que está a jogar. Ele percorre todas as colunas do tabuleiro e verifica se a coluna está vazia na linha superior. Se estiver vazia, adiciona a coluna à lista de movimentos possíveis.
*   **make_move**: Este método executa uma jogada no tabuleiro. Recebe como entrada a coluna (`col`) onde o jogador atual deseja fazer sua jogada. Começa verificando de baixo para cima na coluna para encontrar a primeira posição vazia e, em seguida, coloca a peça do jogador atual nessa posição. Em seguida, alterna para o próximo jogador.
*   **check_winner**: Este método verifica se o jogador atual ganhou o jogo. Ele verifica todas as possíveis linhas, colunas e diagonais para ver se há quatro peças consecutivas do mesmo jogador.
*   **is_board_full**: Este método verifica se o tabuleiro está cheio, ou seja, se não há mais espaços vazios no tabuleiro.
*   **copy**: Este método cria e retorna uma cópia do objeto `GameState`. Isso é útil para fazer cópias do estado do tabuleiro para análise em algoritmos de busca, como o A*.
















In [ ]:
class AStar:
    #constructor of A* class
    def __init__(self, board):
        self.board = board
        self.current_player = PLAYER_X

    #returns the score of a given state
    def get_score(self, state):
        if state.check_winner():
            return 100 if state.current_player == self.current_player else -100
        elif state.is_board_full():
            return 0
        else:
            return 0

    #heuristic value of a given state
    def heuristic(self, state):
        score = 0
        # Horizontal
        for row in state.board:
            for i in range(COLUMNS - 3):
                if all(piece == state.current_player for piece in row[i:i+4]):
                    score += 10

        # Vertical
        for col in range(COLUMNS):
            for i in range(ROWS - 3):
                if all(state.board[row][col] == state.current_player for row in range(i, i+4)):
                    score += 10

        # Diagonal (left to right)
        for row in range(ROWS - 3):
            for col in range(COLUMNS - 3):
                if all(state.board[row+i][col+i] == state.current_player for i in range(4)):
                    score += 10

        # Diagonal (right to left)
        for row in range(ROWS - 3):
            for col in range(3, COLUMNS):
                if all(state.board[row+i][col-i] == state.current_player for i in range(4)):
                    score += 10

        return score
    #returns the best move for the current player in the given state
    def get_best_move(self, state):
        frontier = [(self.get_score(state) + self.heuristic(state), state)]
        explored = set()
        while frontier:
            _, current = min(frontier)
            explored.add(str(current.board))
            if current.check_winner():
                return current.get_legal_moves()[0]
            if not current.get_legal_moves():
                continue
            for move in current.get_legal_moves():
                new_state = current.copy()
                new_state.make_move(move)
                if str(new_state.board) not in explored:
                    frontier.append((self.get_score(new_state) + self.heuristic(new_state), new_state))

        return random.choice(current.get_legal_moves())


##A classe AStarAI implementa um algoritmo de busca A* para encontrar a melhor jogada neste jogo.


---
* __init__: O método de inicialização da classe. Recebe como parâmetro o tabuleiro atual do jogo.
* **et_score**: Retorna a pontuação da situação atual do jogo. Se o jogador atual ganhar, retorna 100. Se o oponente ganhar, retorna -100. Se o jogo empatar, retorna 0.
* **heuristic**: Retorna uma heurística baseada em contagens de peças em sequências de quatro nas linhas, colunas e diagonais do tabuleiro. Quanto mais peças do jogador atual forem encontradas em sequências,consequentemente, maior será a pontuação.
* **get_best_move**: Retorna a melhor jogada possível para o jogador atual, usando o algoritmo A*. Inicialmente, cria uma fronteira contendo o estado atual do jogo e sua pontuação mais a heurística. O algoritmo continua até encontrar um estado em que o jogador atual ganhe ou até que todos os estados possíveis tenham sido explorados. Retorna a primeira jogada do estado final encontrado. Se nenhum estado vencedor for encontrado, retorna uma jogada aleatória entre as legais do estado final.





In [ ]:
class ConnectFourGUI:
    #constructor of ConnectFourGUI class
    def __init__(self, master, ai_type='mcts'):
        self.master = master
        self.master.title("Connect Four")
        for i in range(ROWS + 3):
            self.master.grid_rowconfigure(i, weight=1)
        for j in range(COLUMNS):
            self.master.grid_columnconfigure(j, weight=1)

        self.board = [[EMPTY for _ in range(COLUMNS)] for _ in range(ROWS)]
        self.current_player = PLAYER_X

        self.ai_type = ai_type

        self.create_board_buttons()
        self.create_labels()
        self.create_control_buttons()

        # If current player is O, make move automatically
        if self.current_player == PLAYER_O:
            if self.ai_type == 'mcts':
                self.make_computer_move()
            else:
                self.make_computer_move_astar()

    #creates a frame for control buttons
    def create_control_buttons(self):
        control_frame = tk.Frame(self.master)
        control_frame.grid(row=ROWS + 2, columnspan=COLUMNS, sticky="nsew")

        # AI type selection
        ai_type_label = tk.Label(control_frame, text="Choose AI:")
        ai_type_label.grid(row=0, column=0, padx=(10, 5))

        self.ai_type_var = tk.StringVar(control_frame)
        self.ai_type_var.set(self.ai_type)
        mcts_button = tk.Radiobutton(control_frame, text="MCTS", variable=self.ai_type_var, value='mcts')
        mcts_button.grid(row=0, column=1)

        astar_button = tk.Radiobutton(control_frame, text="A*", variable=self.ai_type_var, value='astar')
        astar_button.grid(row=0, column=2)

        # Restart button
        restart_button = tk.Button(control_frame, text='Restart Game', command=self.restart_game)
        restart_button.grid(row=0, column=3, padx=(20, 10))

    #enables all buttons in the board
    def enable_buttons(self):
        for button in self.buttons:
            button.config(state=tk.NORMAL)

    #restarts the game by resetting everything
    def restart_game(self):
        self.board = [[EMPTY for _ in range(COLUMNS)] for _ in range(ROWS)]
        self.current_player = PLAYER_X
        self.update_board()
        self.enable_buttons()
        self.status_label.config(text="Player X Turn")

    #creates a grid of buttons for the game board and a label for each button
    def create_board_buttons(self):
        self.buttons = []
        for col in range(COLUMNS):
            button = tk.Button(self.master, text=' ', width=4, height=2,
                               command=lambda col=col: self.make_move(col), bg='blue')
            button.grid(row=0, column=col, sticky="nsew")
            button.grid_propagate(False)
            self.buttons.append(button)

        self.game_board = [[None for _ in range(COLUMNS)] for _ in range(ROWS)]
        for row in range(ROWS):
            for col in range(COLUMNS):
                label = tk.Label(self.master, text='', width=4, height=2, relief="sunken", bg='blue')
                label.grid(row=row+1, column=col, sticky="nsew")
                label.grid_propagate(False)
                self.game_board[row][col] = label

    #creates a label for the game status
    def create_labels(self):
        self.status_label = tk.Label(self.master, text="Player X Turn")
        self.status_label.grid(row=ROWS+1, columnspan=COLUMNS)

    #handles a user move when a button is clicked
    def make_move(self, col):
        for row in range(ROWS-1, -1, -1):
            if self.board[row][col] == EMPTY:
                self.board[row][col] = self.current_player
                self.update_board_space(row, col)
                if self.check_winner():
                    self.status_label.config(text=f"Player {self.current_player} wins!")
                    self.disable_buttons()
                elif self.is_board_full():
                    self.status_label.config(text="It's a tie!")
                    self.disable_buttons()
                else:
                    self.current_player = PLAYER_O if self.current_player == PLAYER_X else PLAYER_X
                    if self.current_player == PLAYER_X:
                        self.status_label.config(text="Player X Turn")
                    else:
                        self.status_label.config(text="Player O Turn")
                        if self.ai_type == 'mcts':
                            self.make_computer_move()
                        else:
                            self.make_computer_move_astar()
                break

    #handles a computer move when it is the computer's turn MCST
    def make_computer_move(self):
        root_state = GameState(self.board, self.current_player)
        root_node = CMTS(root_state)
        for _ in range(SIMULATION_COUNT):
            node = root_node
            state = root_state.copy()
            while not node.is_fully_expanded() and not state.check_winner() and not state.is_board_full():
                if not node.is_fully_expanded():
                    child = node.expand()
                    if child:
                        node = child
                        state.make_move(random.choice(state.get_legal_moves()))
                    else:
                        break
            if not state.check_winner() and not state.is_board_full():
                moves = state.get_legal_moves()
                move = random.choice(moves)
                state.make_move(move)
                node = CMTS(state, parent=node)
                node.expand()
            while not state.check_winner() and not state.is_board_full():
                moves = state.get_legal_moves()
                move = random.choice(moves)
                state.make_move(move)

            result = 1 if state.check_winner() and state.current_player == PLAYER_O else 0
            node.backpropagate(result)

        best_child = root_node.select_child(exploration_factor=0)
        if best_child:
            self.board = best_child.state.board
            self.current_player = PLAYER_X if self.current_player == PLAYER_O else PLAYER_O
            self.update_board()

    #handles a computer move when it is the computer's turn A*
    def make_computer_move_astar(self):
        ai = AStar(self.board)
        col = ai.get_best_move(GameState(self.board, self.current_player))
        self.board[ROWS-1][col] = self.current_player
        self.update_board()

    #updates the game board by changing the background color of each button
    def update_board(self):
        for row in range(ROWS):
            for col in range(COLUMNS):
                if self.board[row][col] == PLAYER_X:
                    color = 'red'
                elif self.board[row][col] == PLAYER_O:
                    color = 'yellow'
                else:
                    color = 'blue'
                self.game_board[row][col].config(bg=color)

    #method updates a single button in the game board
    def update_board_space(self, row, col):
        if self.current_player == PLAYER_X:
            color = 'red'
        else:
            color = 'yellow'
        self.game_board[row][col].config(bg=color)

    #method checks if there is a winner
    def check_winner(self):
        # Horizontal
        for row in self.board:
            for i in range(COLUMNS - 3):
                if all(piece == self.current_player for piece in row[i:i+4]):
                    return True

        # Vertical
        for col in range(COLUMNS):
            for i in range(ROWS - 3):
                if all(self.board[row][col] == self.current_player for row in range(i, i+4)):
                    return True

        # Diagonal (left to right)
        for row in range(ROWS - 3):
            for col in range(COLUMNS - 3):
                if all(self.board[row+i][col+i] == self.current_player for i in range(4)):
                    return True

        # Diagonal (right to left)
        for row in range(ROWS - 3):
            for col in range(3, COLUMNS):
                if all(self.board[row+i][col-i] == self.current_player for i in range(4)):
                    return True

        return False

    #checks if the board is full
    def is_board_full(self):
        return all(piece != EMPTY for row in self.board for piece in row)

    #disables all the buttons
    def disable_buttons(self):
        for button in self.buttons:
            button.config(state=tk.DISABLED)



# A classe `ConnectFourGUI` implementa a interface gráfica do jogo Connect Four. Ela gerencia a criação e atualização do tabuleiro, permite que o jogador faça movimentos clicando nos botões, e controla a lógica do jogo, incluindo a análise  de vitória ou empate. Além disso, oferece a opção de jogar contra uma inteligência artificial, utilizando os algoritmos MCTS ou A* para fazer movimentos automáticos do computador.

*   **init**: Este método é chamado quando um objeto da classe é criado. Aqui, configuramos a janela do jogo, criamos o tabuleiro, definimos o jogador atual e inicializamos o tipo de inteligência artificial a ser usado (`ai_type`).
*   **create_board_buttons**: Este métodp cria os botões do tabuleiro do jogo. Cada botão representa uma coluna onde o jogador pode fazer uma jogada.
*  **método create_labels**: Cria os rótulos que exibem o status do jogo, como de quem é a vez ou se houve um vencedor.
*  **create_control_buttons**: Cria os botões de controle, como a opção para escolher o tipo de inteligência artificial (MCTS ou A*) e o botão de reiniciar o jogo.
*   **make_move**: Este método é chamado quando um jogador faz uma jogada. Ele coloca a peça do jogador atual no tabuleiro na coluna escolhida e verifica se houve um vencedor ou se o jogo terminou em empate.
*   **make_computer_move** e **make_computer_move_astar**: Estes métodos são responsáveis por fazer a jogada do computador. Dependendo do tipo de inteligência artificial selecionada, eles chamam o método apropriado para fazer a jogada.
*   **update_board**: Este método atualiza a aparência visual do tabuleiro após cada jogada, alterando as cores dos botões para representar as peças dos jogadores.
*   **check_winner** e **is_board_full** : Estes métodos verificam se houve um vencedor ou se o tabuleiro está cheio, respectivamente.
*   **disable_buttons**: Este método desativa todos os botões do tabuleiro quando o jogo termina.













In [ ]:
def main():
    root = tk.Tk()
    ai_type = tk.StringVar(root)
    ai_type.set('mcts')
    tk.Radiobutton(root, text="MCTS", variable=ai_type, value='mcts').grid(row=ROWS+2, column=0, sticky="w")
    tk.Radiobutton(root, text="A*", variable=ai_type, value='astar').grid(row=ROWS+2, column=1, sticky="w")

    game = ConnectFourGUI(root, ai_type=ai_type.get())
    root.mainloop()

if __name__ == "__main__":
    main()

Esta funcao serve apenas para iniciar o jogo de Connect Four com a interface gráfica.

## **Resultados e algumas observações**:
Com o uso dos algoritmos foi possível criar um perfil de computador inteligente para competir com um humano. Com um número suficiente de simulações, o computador é capaz de tomar decisões inteligentes para maximizar suas chances de vitória. Por outro lado, o desempenho do algoritmo tende a variar com o número de simulações e com o fator exploração.

## **Conclusão**:
Ao longo do desenvolvimento deste projeto, foi possível explorar conceitos importantes de interface gráfica, lógica de jogo e algoritmos de inteligência artificial. A interface gráfica foi projetada de forma simples e intuitiva, garantindo uma experiência de jogo agradável para quem esta a jogar. A lógica de jogo foi implementada de forma inteligente, garantindo que todas as regras deste jogo(Connect Four) fossem respeitadas